## Evaluating the Walkability of the Pedestrian Built-Environment using Geographic Information Systems



### Workbook Objectives

*   Create a function to calculate metrics of each line segment within network
*   Update feature classes attribute data
*   Convert feature class to dataframe
*   Analyze data and convert field data collected metrics
*   Update the Feature Class with the analysis done on the dataframe
*   Export dataframe to csv


In [1]:
#import modules

import arcpy # using ArcGIS Desktop Python 2.7.18
import os
import pandas as pd
import numpy as np
import datetime



Create a function to calculate walkabilty of each line segment within network

In [2]:
# Set the workspace to the relative path of the geodatabase
arcpy.env.workspace = r""

# Feature class and layer names
pointFeatures = "POINTS" # points associated with network that will calculate walkability
pedLine = "" #Network lines 

In [3]:
# Create a function that will update pedestrian network attributes

def updateFeaturePresence(bufferDistance, feature, attributeName):
    """
    Updates an attribute in the pedestrian line feature class based on the spatial relationship with a given feature type.

    :bufferDistance:--->The buffer distance to use for the feature as x Meters. (e.g. 40 Meters)
    :feature:---------->The input feature class (e.g., bins, benches).
    :pedLine:---------->The pedestrian line feature class to update.
    :attributeName:---->The attribute in the pedestrian line feature class to update.
    """
    print("Starting update for {} in {} based on {} with a buffer of {}.".format(attributeName, pedLine, feature, bufferDistance))

    feature_layer = "{}_layer".format(feature)
    arcpy.MakeFeatureLayer_management(in_features = pointFeatures, out_layer = feature_layer, where_clause = "TYPE = '{}'".format(feature))
    print("Feature layer created for {}.".format(feature))


   # Create a buffer around each feature and store it in a temporary feature class
    feature_buffer = os.path.join(arcpy.env.workspace, "Temp{}Buffer".format(feature))
    arcpy.Buffer_analysis(feature_layer, feature_buffer, bufferDistance)
    print("Buffer of {} created for {}.".format(bufferDistance, feature))


    edit = arcpy.da.Editor(arcpy.env.workspace)

    try:
    # Start an edit operation
        edit.startEditing(False, True)
        edit.startOperation()
        print("Edit session started.")
    # Loop through each line in PED_LINE
        countUpdated = 0
        with arcpy.da.UpdateCursor(pedLine, ["SHAPE@", attributeName]) as cursor:
            for row in cursor:
            # Select features within the buffer that intersect the current line
                arcpy.SelectLayerByLocation_management(feature_layer, "INTERSECT", row[0], search_distance=bufferDistance)

            # Check if any features are selected
                if int(arcpy.GetCount_management(feature_layer)[0]) > 0:
                # Update attribute to 'YES' if features are found
                    row[1] = "YES"
                else:
                # Update attribute to 'NO' if no features are found
                    row[1] = "NO"

            # Update the row in the table
                cursor.updateRow(row)
                countUpdated += 1

        edit.stopOperation()
        edit.stopEditing(True)
        print("Edit operation completed. Number of rows updated: {}".format(countUpdated))

    except Exception as e:
        # If an error occurred, stop the edit operation and report the error
        edit.stopOperation()
        print("An error occurred: " + str(e))
        # Clean up: delete the temporary buffer and feature layer
    
        print("Cleanup completed for {}.".format(feature))

    print("Update completed for {}.".format(attributeName))

Update feature classes attribute data

In [4]:
# set parameters for buffer

# updateFeaturePresence(bufferDistance, feature, pedLine, attributeName)
binBufferDistance = "20 Meters"
benchBufferDistance = "20 Meters"
musterBufferDistance = "70 Meters"
phoneBufferDistance = "50 Meters"
cctvBufferDistance = "30 Meters"
shuttleBufferDistance = "100 Meters"

#Use function to update Feature classes

updateFeaturePresence(binBufferDistance,"BIN","BIN")
updateFeaturePresence(benchBufferDistance,"BENCH","BENCHES")
updateFeaturePresence(cctvBufferDistance,"SURVEILLANCE","SURV")
updateFeaturePresence(musterBufferDistance,"MUSTER_POINT","MUSTER")
updateFeaturePresence(phoneBufferDistance,"EMERGENCY_PHONE","E_PHONES")
updateFeaturePresence(shuttleBufferDistance,"SHUTTLESTOP","SHUT_STOP")

Starting update for BIN in Ped_Line_Eng based on BIN with a buffer of 20 Meters.
Feature layer created for BIN.
Buffer of 20 Meters created for BIN.
Edit session started.
Edit operation completed. Number of rows updated: 178
Update completed for BIN.
Starting update for BENCHES in Ped_Line_Eng based on BENCH with a buffer of 20 Meters.
Feature layer created for BENCH.
Buffer of 20 Meters created for BENCH.
Edit session started.
Edit operation completed. Number of rows updated: 178
Update completed for BENCHES.
Starting update for SURV in Ped_Line_Eng based on SURVEILLANCE with a buffer of 30 Meters.
Feature layer created for SURVEILLANCE.
Buffer of 30 Meters created for SURVEILLANCE.
Edit session started.
Edit operation completed. Number of rows updated: 178
Update completed for SURV.
Starting update for MUSTER in Ped_Line_Eng based on MUSTER_POINT with a buffer of 70 Meters.
Feature layer created for MUSTER_POINT.
Buffer of 70 Meters created for MUSTER_POINT.
Edit session started.
Edi

Convert feature class to pandas dataframe

In [5]:
gdbPath = r""
featureClass = "" # The lines feature class wishing to convert
# Ped_Line_Eng
pedLine = os.path.join(gdbPath,featureClass)

# print(pedLine)
fields = [field.name for field in arcpy.ListFields(pedLine)]

pedData = []

with arcpy.da.SearchCursor(pedLine,fields) as cursor:
    for row in cursor:
        pedData.append(row)


pedDataFrame = pd.DataFrame(pedData,columns=fields)

Analyze data and convert field data collected metrics these metric guidelines were provided by a project to assign each value so that the metric could be ultimately be used to calculate Walkability

In [6]:
#Read dataframe 

pedDataFrame.head()

,OBJECTID,SHAPE,P_ID,PED_TYPE,GRADIENT,MATERIAL,CONDITION,CLEAN,CURB_DIST,WIDTH,...,SAFETY,CON_SCORE,DEST_SCORE,AES_SCORE,SAFE_TRAF,WALK_SCORE,WALK_TSCORE,PED_NAME,SHAPE_Length,WALK_CATEGORY
0,1,"(675088.1771, 1176404.36765)",1000,4,F,A,F,G,<1.5,>1.5 | <3,...,1.5,0.0,1.0,4.0,None,6.5,1.73,CROSSWALK,6.932738,None
1,2,"(675088.62285, 1176410.4672)",1001,2,F,C,G,G,<1.5,<1.5,...,1.5,1.0,0.0,3.0,None,5.5,1.35,SIDEWALK,5.299458,None
2,3,"(675084.9992, 1176413.82015)",1002,2,F,C,G,G,>1.5,<1.5,...,0.5,1.0,0.0,2.5,None,4.0,0.77,SIDEWALK,7.822507,None
3,4,"(675078.091283, 1176416.73878)",1003,5,F,A,G,G,None,<1.5,...,0.5,2.0,1.0,2.5,None,6.0,1.54,MISSING CONNECTION,7.552591,None
4,5,"(675071.558007, 1176418.15565)",1004,2,F,C,G,G,>1.5,<1.5,...,2.0,1.0,1.0,2.0,None,6.0,1.54,SIDEWALK,7.333319,None


In [7]:
# Kerb Distance 

# pedDataFrame[["CURB_DIST","KERB_NUM"]]
pedDataFrame["KERB_NUM"] = pedDataFrame["KERB_NUM"].astype(float)
# pedDataFrame["KERB_NUM"].dtypes

def calcKerbNum(CURB_DIST):
    if CURB_DIST == "0":
        return 2
    elif CURB_DIST == "<1.5":
        return 1
    elif CURB_DIST == ">1.5":
        return 0
    else:
        return np.nan


pedDataFrame["KERB_NUM"] = pedDataFrame["CURB_DIST"].apply(calcKerbNum)


pedDataFrame[["CURB_DIST","KERB_NUM"]].head()

,CURB_DIST,KERB_NUM
0,<1.5,1.0
1,<1.5,1.0
2,>1.5,0.0
3,None,NaN
4,>1.5,0.0


In [8]:
# Traffic Speed 

# pedDataFrame[["CURB_DIST","KERB_NUM"]]
pedDataFrame["SPEED_NUM"] = pedDataFrame["SPEED_NUM"].astype(float)
# pedDataFrame["KERB_NUM"].dtypes

def calcSpeedNum(T_SPEED):
    if T_SPEED == "10":
        return 0
    elif T_SPEED == "30" or T_SPEED == "OTHER":
        return 0.5
    else:
        return np.nan


pedDataFrame["SPEED_NUM"] = pedDataFrame["T_SPEED"].apply(calcSpeedNum)


pedDataFrame[["T_SPEED","SPEED_NUM"]].head()

,T_SPEED,SPEED_NUM
0,30,0.5
1,30,0.5
2,30,0.5
3,10,0.0
4,30,0.5


In [9]:
# Buffer Present beside roadway

# pedDataFrame[["CURB_DIST","KERB_NUM"]]
pedDataFrame["BUFF_NUM"] = pedDataFrame["BUFF_NUM"].astype(float)
# pedDataFrame["KERB_NUM"].dtypes

def calcBuffNum(BUF_PRES):
    if BUF_PRES == "NO":
        return 1
    elif BUF_PRES == 'YES':
        return 0
    else:
        return np.nan


pedDataFrame["BUFF_NUM"] = pedDataFrame["BUF_PRES"].apply(calcBuffNum)


pedDataFrame[["BUF_PRES","BUFF_NUM"]].head()

,BUF_PRES,BUFF_NUM
0,N/A,NaN
1,YES,0.0
2,YES,0.0
3,YES,0.0
4,YES,0.0


In [10]:
# Presence of a Traffic Control Device

# pedDataFrame[["CURB_DIST","KERB_NUM"]]
pedDataFrame["TCD_NUM"] = pedDataFrame["TCD_NUM"].astype(float)
# pedDataFrame["KERB_NUM"].dtypes

def calcTCDNum(TCDPRES):
    if TCDPRES == "NO":
        return 1
    elif TCDPRES == 'YES':
        return 0
    else:
        return np.nan


pedDataFrame["TCD_NUM"] = pedDataFrame["TCDPRES"].apply(calcTCDNum)


pedDataFrame[["TCDPRES","TCD_NUM"]].head()

,TCDPRES,TCD_NUM
0,N/A,NaN
1,YES,0.0
2,YES,0.0
3,YES,0.0
4,NO,1.0


In [11]:
# Presence of Lighting 

# pedDataFrame[["LIGHTING","LIGHT_NUM"]]
pedDataFrame["LIGHT_NUM"] = pedDataFrame["LIGHT_NUM"].astype(float)

def calcLightNum(LIGHTING):
    if LIGHTING == "P":
        return 0
    elif LIGHTING == "R" or LIGHTING == "O":
        return 0.5
    elif LIGHTING == "N":
        return 1

pedDataFrame["LIGHT_NUM"] = pedDataFrame["LIGHTING"].apply(calcLightNum)

pedDataFrame[["LIGHTING","LIGHT_NUM"]].head(10)



,LIGHTING,LIGHT_NUM
0,P,0.0
1,P,0.0
2,P,0.0
3,R,0.5
4,R,0.5
5,R,0.5
6,R,0.5
7,R,0.5
8,R,0.5
9,P,0.0


In [12]:
# CCTV Presence 

# pedDataFrame[["SURV","SURV_NUM"]]
# pedDataFrame["SURV_NUM"] = pedDataFrame["SURV_NUM"].astype(float)

def calcSurvNum(SURV):
    if SURV == "YES":
        return 0
    elif SURV == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["SURV_NUM"] = pedDataFrame["SURV"].apply(calcSurvNum)

pedDataFrame[["SURV","SURV_NUM"]].head(10)


,SURV,SURV_NUM
0,YES,0
1,YES,0
2,YES,0
3,YES,0
4,YES,0
5,NO,1
6,YES,0
7,NO,1
8,NO,1
9,NO,1


In [13]:
# Muster Points 

# pedDataFrame[["MUSTER","MUST_NUM"]]
pedDataFrame["MUST_NUM"] = pedDataFrame["MUST_NUM"].astype(float)

def calcMustNum(MUSTER):
    if MUSTER == "YES":
        return 0
    elif MUSTER == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["MUST_NUM"] = pedDataFrame["MUSTER"].apply(calcMustNum)

pedDataFrame[["MUSTER","MUST_NUM"]].head(10)

,MUSTER,MUST_NUM
0,YES,0
1,YES,0
2,YES,0
3,YES,0
4,YES,0
5,YES,0
6,YES,0
7,YES,0
8,YES,0
9,NO,1


In [14]:
# Emergency Phone 

# pedDataFrame[["EPHONE_NUM","EPHONE_NUM"]]
pedDataFrame["EPHONE_NUM"] = pedDataFrame["EPHONE_NUM"].astype(float)

def calcPhoneNum(E_PHONES):
    if E_PHONES == "YES":
        return 0
    elif E_PHONES == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["EPHONE_NUM"] = pedDataFrame["E_PHONES"].apply(calcPhoneNum)

pedDataFrame[["E_PHONES","EPHONE_NUM"]].head(10)

,E_PHONES,EPHONE_NUM
0,YES,0
1,YES,0
2,YES,0
3,YES,0
4,YES,0
5,NO,1
6,YES,0
7,NO,1
8,YES,0
9,NO,1


In [15]:
# Wayfinder Presence 

pedDataFrame["WFIND_NUM"] = pedDataFrame["WFIND_NUM"].astype(float)

def calcWayFindNum(WAY_FI):
    if WAY_FI == "Y":
        return 0
    elif WAY_FI == "N":
        return 1
    else:
        return np.nan


pedDataFrame["WFIND_NUM"] = pedDataFrame["WAY_FI"].apply(calcWayFindNum)

pedDataFrame[["WAY_FI","WFIND_NUM"]].head(10)

,WAY_FI,WFIND_NUM
0,Y,0
1,Y,0
2,Y,0
3,N,1
4,Y,0
5,Y,0
6,Y,0
7,N,1
8,Y,0
9,N,1


In [16]:
# Walkway gradient 

pedDataFrame["GRAD_NUM"] = pedDataFrame["GRAD_NUM"].astype(float)

def calcGradNum(GRADIENT):
    if GRADIENT == "F":
        return 0
    elif GRADIENT == "SL":
        return 1
    elif GRADIENT == "ST":
        return 2    
    else:
        return np.nan


pedDataFrame["GRAD_NUM"] = pedDataFrame["GRADIENT"].apply(calcGradNum)

pedDataFrame[["GRADIENT","GRAD_NUM"]].head(10)

,GRADIENT,GRAD_NUM
0,F,0.0
1,F,0.0
2,F,0.0
3,F,0.0
4,F,0.0
5,F,0.0
6,F,0.0
7,F,0.0
8,F,0.0
9,F,0.0


In [17]:
#Walway Material 

pedDataFrame["MAT_NUM"] = pedDataFrame["MAT_NUM"].astype(float)

def calcMatNum(MATERIAL):
    if MATERIAL == "A" or MATERIAL == "C":
        return 0
    elif MATERIAL == "P":
        return 1
    elif MATERIAL == "D/S":
        return 2    
    elif MATERIAL == "G":
        return 3    
    else:
        return np.nan


pedDataFrame["MAT_NUM"] = pedDataFrame["MATERIAL"].apply(calcMatNum)

pedDataFrame[["MATERIAL","MAT_NUM"]].head(10)
# pedDataFrame["MAT_NUM"].value_counts()

,MATERIAL,MAT_NUM
0,A,0.0
1,C,0.0
2,C,0.0
3,A,0.0
4,C,0.0
5,O,NaN
6,C,0.0
7,C,0.0
8,C,0.0
9,A,0.0


In [18]:
# Number of Kerb Cuts on walkway

pedDataFrame["KERB_CUT_NUM"] = pedDataFrame["KERB_CUT_NUM"].astype(float)

def calcKerbNum(CURB_CUTS):
    if CURB_CUTS >= 2:
        return 0
    elif CURB_CUTS == 0:
        return 1
    elif CURB_CUTS == 1:
        return 0.5
    else:
        return np.nan


pedDataFrame["KERB_CUT_NUM"] = pedDataFrame["CURB_CUTS"].apply(calcKerbNum)

pedDataFrame[["CURB_CUTS","KERB_CUT_NUM"]].head(10)
# pedDataFrame["MAT_NUM"].value_counts()

,CURB_CUTS,KERB_CUT_NUM
0,0,1.0
1,0,1.0
2,0,1.0
3,0,1.0
4,0,1.0
5,0,1.0
6,0,1.0
7,0,1.0
8,0,1.0
9,0,1.0


In [19]:
# Walkway Width

pedDataFrame["WID_NUM"] = pedDataFrame["WID_NUM"].astype(float)

def calcwWidthNum(WIDTH):
    if WIDTH == "<1.5":
        return 1
    elif WIDTH == ">1.5 | <3" or WIDTH == ">3":
        return 0    
    else:
        return np.nan


pedDataFrame["WID_NUM"] = pedDataFrame["WIDTH"].apply(calcwWidthNum)

pedDataFrame[["WIDTH","WID_NUM"]].head(10)
# pedDataFrame["MAT_NUM"].value_counts()

,WIDTH,WID_NUM
0,>1.5 | <3,0
1,<1.5,1
2,<1.5,1
3,<1.5,1
4,<1.5,1
5,<1.5,1
6,<1.5,1
7,<1.5,1
8,<1.5,1
9,<1.5,1


In [20]:
#OBSTRUCTION PRESENCE ON WALKWAY [need to review Geodata]

pedDataFrame["OBS_NUM"] = pedDataFrame["OBS_NUM"].astype(float)

def calcObsNum(OBST_PRES):
    if OBST_PRES == "N":
        return 0
    elif OBST_PRES == "Y":
        return 1
    else:
        return np.nan


pedDataFrame["OBS_NUM"] = pedDataFrame["OBST_PRES"].apply(calcObsNum)

pedDataFrame[["OBST_PRES","OBS_NUM"]].head(10)

# pedDataFrame["OBS_NUM"].value_counts()

,OBST_PRES,OBS_NUM
0,N,0
1,N,0
2,N,0
3,N,0
4,N,0
5,N,0
6,N,0
7,N,0
8,Y,1
9,N,0


In [21]:
# Walkway Completeness

pedDataFrame["COMP_NUM"] = pedDataFrame["COMP_NUM"].astype(float)

def calcCompNum(COMPL):
    if COMPL == "Y":
        return 0
    elif COMPL == "N":
        return 1
    else:
        return np.nan


pedDataFrame["COMP_NUM"] = pedDataFrame["COMPL"].apply(calcCompNum)

pedDataFrame[["COMPL","COMP_NUM"]].head(10)

,COMPL,COMP_NUM
0,Y,0
1,Y,0
2,Y,0
3,Y,0
4,Y,0
5,N,1
6,N,1
7,N,1
8,Y,0
9,Y,0


In [22]:
# Walkway connectivity 

pedDataFrame["CONN_NUM"] = pedDataFrame["CONN_NUM"].astype(float)

def calcConnNum(CONN):
    if CONN >= 3:
        return 0
    elif CONN == 1 or CONN == 2:
        return 1
    else:
        return np.nan


pedDataFrame["CONN_NUM"] = pedDataFrame["CONN"].apply(calcConnNum)

# pedDataFrame[["CONN","CONN_NUM"]].head(10)

pedDataFrame["CONN_NUM"].value_counts()

0    143
1     35
Name: CONN_NUM, dtype: int64

In [23]:
# Shuttle Bus Stop Presence 


pedDataFrame["SHUT_NUM"] = pedDataFrame["SHUT_NUM"].astype(float)

def calcCompNum(SHUT_STOP):
    if SHUT_STOP == "YES":
        return 0
    elif SHUT_STOP == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["SHUT_NUM"] = pedDataFrame["SHUT_STOP"].apply(calcCompNum)

pedDataFrame[["SHUT_STOP","SHUT_NUM"]].head(10)

,SHUT_STOP,SHUT_NUM
0,YES,0
1,YES,0
2,YES,0
3,YES,0
4,YES,0
5,NO,1
6,YES,0
7,NO,1
8,NO,1
9,NO,1


In [24]:
# Walkway cleanliness

pedDataFrame["CLEAN_NUM"] = pedDataFrame["CLEAN_NUM"].astype(float)

def calcCleanNum(CLEAN):
    if CLEAN == "G":
        return 0
    elif CLEAN == "F":
        return 1
    elif CLEAN == "P":
        return 3    
    elif CLEAN == "UR":
        return 4
    else:
        return np.nan


pedDataFrame["CLEAN_NUM"] = pedDataFrame["CLEAN"].apply(calcCleanNum)

pedDataFrame[["CLEAN","CLEAN_NUM"]].head(10)

,CLEAN,CLEAN_NUM
0,G,0
1,G,0
2,G,0
3,G,0
4,G,0
5,G,0
6,G,0
7,G,0
8,G,0
9,G,0


In [25]:
#Shelter presence

pedDataFrame["SHELT_NUM"] = pedDataFrame["SHELT_NUM"].astype(float)

def calcSheltNum(SHELTER):
    if SHELTER == "Y":
        return 0
    elif SHELTER == "N":
        return 1
    else:
        return np.nan


pedDataFrame["SHELT_NUM"] = pedDataFrame["SHELTER"].apply(calcSheltNum)

pedDataFrame[["SHELTER","SHELT_NUM"]].head(10)

,SHELTER,SHELT_NUM
0,N,1
1,N,1
2,N,1
3,N,1
4,N,1
5,N,1
6,N,1
7,N,1
8,N,1
9,N,1


In [26]:
# Bins presence 

pedDataFrame["BINS_NUM"] = pedDataFrame["BINS_NUM"].astype(float)

def calcBinNum(BIN):
    if BIN == "YES":
        return 0
    elif BIN == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["BINS_NUM"] = pedDataFrame["BIN"].apply(calcBinNum)

pedDataFrame[["BIN","BINS_NUM"]].head(10)

,BIN,BINS_NUM
0,YES,0
1,YES,0
2,YES,0
3,YES,0
4,YES,0
5,NO,1
6,NO,1
7,YES,0
8,NO,1
9,YES,0


In [27]:
# Bench presence 

pedDataFrame["BENCH_NUM"] = pedDataFrame["BENCH_NUM"].astype(float)

def calcBenchNum(BENCHES):
    if BENCHES == "YES":
        return 0
    elif BENCHES == "NO":
        return 1
    else:
        return np.nan


pedDataFrame["BENCH_NUM"] = pedDataFrame["BENCHES"].apply(calcBenchNum)

pedDataFrame[["BENCHES","BENCH_NUM"]].head(10)

,BENCHES,BENCH_NUM
0,NO,1
1,NO,1
2,NO,1
3,NO,1
4,NO,1
5,NO,1
6,NO,1
7,YES,0
8,NO,1
9,NO,1


In [28]:
# Trees presence

pedDataFrame["TREE_NUM"] = pedDataFrame["TREE_NUM"].astype(float)

def calcTreeNum(TREES):
    if TREES == "S" or TREES == 'D':
        return 0
    elif TREES == "VF":
        return 0.5
    elif TREES == "N":
        return 1    
    else:
        return np.nan


pedDataFrame["TREE_NUM"] = pedDataFrame["TREES"].apply(calcTreeNum)

pedDataFrame[["TREES","TREE_NUM"]].head(10)

,TREES,TREE_NUM
0,N,1.0
1,N,1.0
2,VF,0.5
3,VF,0.5
4,D,0.0
5,N,1.0
6,S,0.0
7,VF,0.5
8,N,1.0
9,N,1.0


In [29]:
# Condition of walkway

pedDataFrame["COND_NUM"] = pedDataFrame["COND_NUM"].astype(float)

def calcCondNum(CONDITION):
    if CONDITION == "G":
        return 0
    elif CONDITION == "F":
        return 1
    elif CONDITION == "P":
        return 3    
    elif CONDITION == "UR":
        return 4
    else:
        return np.nan


pedDataFrame["COND_NUM"] = pedDataFrame["CONDITION"].apply(calcCondNum)

pedDataFrame[["CONDITION","COND_NUM"]].head(10)

,CONDITION,COND_NUM
0,F,1
1,G,0
2,G,0
3,G,0
4,G,0
5,G,0
6,G,0
7,P,3
8,G,0
9,G,0


In [30]:
# Calucate the Traffic Score for the nnetwork

# pedDataFrame[['PED_NAME','KERB_NUM','SPEED_NUM','BUFF_NUM','TCD_NUM','TRAF_SCORE']].head(30)
pedDataFrame["TRAF_SCORE"] = pedDataFrame["TRAF_SCORE"].astype(float)

trafScore = pedDataFrame['PED_NAME'].isin(['CROSSWALK', 'SIDEWALK', 'MISSING CONNECTION'])

trafSum = ['KERB_NUM','SPEED_NUM','BUFF_NUM','TCD_NUM']

# trafScore['TRAF_SCORE'] = trafScore[trafSum].sum(axis=1,skipna=True)

pedDataFrame.loc[trafScore,'TRAF_SCORE'] = pedDataFrame.loc[trafScore, trafSum].sum(axis=1, skipna=True)


pedDataFrame[['OBJECTID','PED_NAME','KERB_NUM','SPEED_NUM','BUFF_NUM','TCD_NUM','TRAF_SCORE']].head(10)

,OBJECTID,PED_NAME,KERB_NUM,SPEED_NUM,BUFF_NUM,TCD_NUM,TRAF_SCORE
0,1,CROSSWALK,1.0,0.5,NaN,NaN,1.5
1,2,SIDEWALK,1.0,0.5,0.0,0.0,1.5
2,3,SIDEWALK,0.0,0.5,0.0,0.0,0.5
3,4,MISSING CONNECTION,NaN,0.0,0.0,0.0,0.0
4,5,SIDEWALK,0.0,0.5,0.0,1.0,1.5
5,6,MISSING CONNECTION,1.0,0.5,0.0,0.0,1.5
6,7,SIDEWALK,2.0,0.5,1.0,1.0,4.5
7,8,SIDEWALK,0.0,0.5,0.0,0.0,0.5
8,9,SIDEWALK,2.0,0.5,1.0,1.0,4.5
9,10,MISSING CONNECTION,NaN,0.5,1.0,1.0,2.5


In [31]:
# Calculate the Individual Security Score for the nnetwork


pedDataFrame["PER_SCORE"] = pedDataFrame["PER_SCORE"].astype(float)
pedDataFrame['PER_SCORE'] = np.nan
pedDataFrame['PER_SCORE'] = pedDataFrame[['LIGHT_NUM','SURV_NUM','MUST_NUM','EPHONE_NUM']].sum(axis = 1,skipna=True)

pedDataFrame[['OBJECTID','PED_NAME','LIGHT_NUM','SURV_NUM','MUST_NUM','EPHONE_NUM','PER_SCORE']].head(15)



,OBJECTID,PED_NAME,LIGHT_NUM,SURV_NUM,MUST_NUM,EPHONE_NUM,PER_SCORE
0,1,CROSSWALK,0.0,0,0,0,0.0
1,2,SIDEWALK,0.0,0,0,0,0.0
2,3,SIDEWALK,0.0,0,0,0,0.0
3,4,MISSING CONNECTION,0.5,0,0,0,0.5
4,5,SIDEWALK,0.5,0,0,0,0.5
5,6,MISSING CONNECTION,0.5,1,0,1,2.5
6,7,SIDEWALK,0.5,0,0,0,0.5
7,8,SIDEWALK,0.5,1,0,1,2.5
8,9,SIDEWALK,0.5,1,0,0,1.5
9,10,MISSING CONNECTION,0.0,1,1,1,3.0


In [32]:
#Calculate Universal Score

pedDataFrame["UNI_SCORE"] = pedDataFrame["UNI_SCORE"].astype(float)
pedDataFrame['UNI_SCORE'] = np.nan
pedDataFrame['UNI_SCORE'] = pedDataFrame[['WFIND_NUM','GRAD_NUM','MAT_NUM','KERB_CUT_NUM','WID_NUM']].sum(axis = 1,skipna=True)

pedDataFrame[['OBJECTID','PED_NAME','WFIND_NUM','GRAD_NUM','MAT_NUM','KERB_CUT_NUM','WID_NUM','UNI_SCORE']].head(15)



,OBJECTID,PED_NAME,WFIND_NUM,GRAD_NUM,MAT_NUM,KERB_CUT_NUM,WID_NUM,UNI_SCORE
0,1,CROSSWALK,0,0.0,0.0,1.0,0,1.0
1,2,SIDEWALK,0,0.0,0.0,1.0,1,2.0
2,3,SIDEWALK,0,0.0,0.0,1.0,1,2.0
3,4,MISSING CONNECTION,1,0.0,0.0,1.0,1,3.0
4,5,SIDEWALK,0,0.0,0.0,1.0,1,2.0
5,6,MISSING CONNECTION,0,0.0,NaN,1.0,1,2.0
6,7,SIDEWALK,0,0.0,0.0,1.0,1,2.0
7,8,SIDEWALK,1,0.0,0.0,1.0,1,3.0
8,9,SIDEWALK,0,0.0,0.0,1.0,1,2.0
9,10,MISSING CONNECTION,1,0.0,0.0,1.0,1,3.0


In [33]:
# Calculate Connectivity 

pedDataFrame["CON_SCO"] = pedDataFrame["CON_SCO"].astype(float)
pedDataFrame['CON_SCO'] = np.nan
pedDataFrame['CON_SCO'] = pedDataFrame[['OBS_NUM','COMP_NUM']].sum(axis = 1,skipna=True)

# pedDataFrame[['OBJECTID','PED_NAME','OBS_NUM','COMP_NUM','CON_SCO']].head(15)

pedDataFrame['CON_SCO'].value_counts()

0    155
1     22
2      1
Name: CON_SCO, dtype: int64

In [34]:
# Calculate Facilities

pedDataFrame["FAC_SCORE"] = pedDataFrame["FAC_SCORE"].astype(float)
pedDataFrame['FAC_SCORE'] = np.nan
pedDataFrame['FAC_SCORE'] = pedDataFrame[['CONN_NUM','SHUT_NUM']].sum(axis = 1,skipna=True)

# pedDataFrame[['OBJECTID','PED_NAME','CONN_NUM','SHUT_NUM','FAC_SCORE']].head(15)

pedDataFrame['FAC_SCORE'].value_counts()


1    100
0     51
2     27
Name: FAC_SCORE, dtype: int64

In [35]:
# Calculate Comfort Score

pedDataFrame["COM_SCORE"] = pedDataFrame["COM_SCORE"].astype(float)
pedDataFrame['COM_SCORE'] = np.nan
pedDataFrame['COM_SCORE'] = pedDataFrame[['CLEAN_NUM','SHELT_NUM','BINS_NUM','BENCH_NUM','TREE_NUM','COND_NUM']].sum(axis = 1,skipna=True)

pedDataFrame[['OBJECTID','PED_NAME','CLEAN_NUM','SHELT_NUM','BINS_NUM','BENCH_NUM','TREE_NUM','COND_NUM','COM_SCORE']].head(15)



,OBJECTID,PED_NAME,CLEAN_NUM,SHELT_NUM,BINS_NUM,BENCH_NUM,TREE_NUM,COND_NUM,COM_SCORE
0,1,CROSSWALK,0,1,0,1,1.0,1,4.0
1,2,SIDEWALK,0,1,0,1,1.0,0,3.0
2,3,SIDEWALK,0,1,0,1,0.5,0,2.5
3,4,MISSING CONNECTION,0,1,0,1,0.5,0,2.5
4,5,SIDEWALK,0,1,0,1,0.0,0,2.0
5,6,MISSING CONNECTION,0,1,1,1,1.0,0,4.0
6,7,SIDEWALK,0,1,1,1,0.0,0,3.0
7,8,SIDEWALK,0,1,0,0,0.5,3,4.5
8,9,SIDEWALK,0,1,1,1,1.0,0,4.0
9,10,MISSING CONNECTION,0,1,0,1,1.0,0,3.0


In [36]:
# Calculate Pedestrian Safety Rating

pedDataFrame["SAFETY"] = pedDataFrame["SAFETY"].astype(float)

pedDataFrame['SAFETY'] = np.nan

def calcSafety(row):
    if row['PED_NAME'] == 'PEDESTRIAN STREET':
        return row['PER_SCORE']
    else:
        return (row['TRAF_SCORE'] + row['PER_SCORE'])


pedDataFrame['SAFETY'] = pedDataFrame.apply(calcSafety,axis=1)

# pedDataFrame[['OBJECTID','PED_NAME','PER_SCORE','TRAF_SCORE','SAFETY']].head(5)
# pedDataFrame.columns

pedDataFrame['SAFETY'].value_counts()

2.0    71
3.0    50
4.0    12
1.0    10
6.0    10
5.0     6
2.5     5
7.0     4
0.5     4
1.5     3
3.5     1
5.5     1
Name: SAFETY, dtype: int64

In [37]:
# Calculate Conspicuous Rating

pedDataFrame["CON_SCORE"] = pedDataFrame["CON_SCORE"].astype(float)

pedDataFrame['CON_SCORE'] = np.nan

pedDataFrame['CON_SCORE'] = (pedDataFrame['UNI_SCORE'] + pedDataFrame['CON_SCO'])


# pedDataFrame[['OBJECTID','PED_NAME','CON_SCO','UNI_SCORE','CON_SCORE']].head(5)
# pedDataFrame.columns

pedDataFrame['CON_SCORE'].value_counts()

1.0    77
2.0    53
3.0    31
4.0     8
1.5     3
0.5     3
6.0     2
5.0     1
Name: CON_SCORE, dtype: int64

In [38]:
# Calculate Destination Rating

pedDataFrame["DEST_SCORE"] = pedDataFrame["DEST_SCORE"].astype(float)

pedDataFrame['DEST_SCORE'] = np.nan

pedDataFrame['DEST_SCORE'] = (pedDataFrame['FAC_SCORE'])

# pedDataFrame[['OBJECTID','PED_NAME','FAC_SCORE','DEST_SCORE']]

pedDataFrame['DEST_SCORE'].value_counts()

1    100
0     51
2     27
Name: DEST_SCORE, dtype: int64

In [39]:
# Calculate Aesthetics Rating

pedDataFrame["AES_SCORE"] = pedDataFrame["AES_SCORE"].astype(float)

pedDataFrame['AES_SCORE'] = np.nan

pedDataFrame['AES_SCORE'] = (pedDataFrame['COM_SCORE'])

# pedDataFrame[['OBJECTID','PED_NAME','COM_SCORE','AES_SCORE']].head(15)

pedDataFrame['AES_SCORE'].value_counts()

3.0    70
4.0    40
2.0    29
1.0    17
3.5     5
5.0     4
2.5     4
4.5     3
6.0     2
7.0     1
5.5     1
0.5     1
1.5     1
Name: AES_SCORE, dtype: int64

In [40]:
# Calculate Walkability Rating 

pedDataFrame["WALK_SCORE"] = pedDataFrame["WALK_SCORE"].astype(float)

pedDataFrame['WALK_SCORE'] = np.nan

pedDataFrame['WALK_SCORE'] = pedDataFrame[['SAFETY','CON_SCORE','DEST_SCORE','AES_SCORE']].sum(axis=1).round(2)


# Normalize Column Data
minWalk = pedDataFrame['WALK_SCORE'].min()
maxWalk = pedDataFrame['WALK_SCORE'].max()


pedDataFrame['WALK_TSCORE'] = (0 + ((pedDataFrame['WALK_SCORE'] - minWalk) * (5 - 0)) / (maxWalk - minWalk)).round(2)


# pedDataFrame['WALK_TSCORE'].value_counts()

In [41]:
#Function that converts to Walkability rating to grading system
def calcWalkability(walk_score):
    if 0 <= walk_score < 1:
        return 'EXCELLENT'
    elif 1 <= walk_score < 2:
        return 'GOOD'
    elif 2 <= walk_score < 3:
        return 'FAIR'
    elif 3 <= walk_score < 4:
        return 'POOR'
    elif 4 <= walk_score < 5:
        return 'VERY POOR'



pedDataFrame["WALK_CATEGORY"] = pedDataFrame["WALK_TSCORE"].apply(calcWalkability)

pedDataFrame["WALK_CATEGORY"].value_counts()

GOOD         81
FAIR         41
POOR         23
EXCELLENT    20
VERY POOR    12
Name: WALK_CATEGORY, dtype: int64

Update the Feature Class with the analysis done on the dataframe

In [42]:
pedDict = pedDataFrame.set_index('P_ID').to_dict(orient='index')

fields = [field.name for field in arcpy.ListFields(pedLine) if field.name != 'P_ID'] + ['P_ID']

edit = arcpy.da.Editor(gdbPath)
edit.startEditing(False, True)
edit.startOperation()

try:
    with arcpy.da.UpdateCursor(pedLine, fields) as cursor:
        for row in cursor:
            p_id = row[-1]  # Get the P_ID from the row
            if p_id in pedDict:
                # Update the row with the corresponding data from the DataFrame
                for i, field in enumerate(fields[:-1]):  # Exclude P_ID from the update
                    row[i] = pedDict[p_id][field]
                cursor.updateRow(row)
    edit.startOperation()
    edit.stopEditing(True)

except Exception as e:
    edit.stopOperation()
    edit.stopEditing(False)
    raise e

Export dataframe to csv

In [43]:
# pedDataFrame.head(20)
current_date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


path = r"path_to_save\{}_pedDataiframe.csv".format(current_date)

pedDataFrame.to_csv(path)

Export feature class to shapefile

In [47]:
current_date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

shapePath = r"E:\GIS\Projects\Walkability\v1\Shapefile"
pedShapefile = r"{}_Ped_Line.shp".format(current_date)

arcpy.FeatureClassToFeatureClass_conversion(pedLine,shapePath, pedShapefile)



<Result 'E:\\GIS\\Projects\\Walkability\\v1\\Shapefile\\20231221_135959_Ped_Line.shp'>